In [1]:
import pandas as pd
import openai
import json
import time
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for the progress bar
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [2]:
#pip install openai

In [3]:
from openai import OpenAI

In [4]:
openai.__version__


'1.1.1'

### Read in annotated data

In [5]:
df = pd.read_csv("../data/prodigy/annotated_output/final/full_combined_dataset_1996.csv", index_col=0)

In [6]:
df.head()

,pmid,journal_name,title,abstract,accepted_label
0,18379746,Der Radiologe,[Pediatric stroke].,Stroke in childhood has gained increasingly mo...,Non-systematic-review
1,24660674,Journal of consulting and clinical psychology,Treatment engagement and response to CBT among...,"In the current study, we compared measures of ...",Human-RCT-non-drug-intervention
2,20159133,Archives of physical medicine and rehabilitation,Relationship between perceived exertion and ph...,To investigate the strength of the relationshi...,Remaining
3,11781147,Biochimica et biophysica acta,Characterization of a missense mutation at his...,Genetic defects in pyruvate dehydrogenase comp...,Remaining
4,31706919,Epilepsy & behavior : E&B,The role of P-glycoprotein (P-gp) and inwardly...,Sudden unexpected death in epilepsy (SUDEP) is...,Non-systematic-review


In [7]:
# Combine the columns
# Implementing custom tags for the combination of journal name, title, and abstract
df['input_journal_title_abstract'] = '<journal>' + df['journal_name'] + '</journal>' + \
                                     '<title>' + df['title'] + '</title>' + \
                                     '<abstract>' + df['abstract'] + '</abstract>'

# Implementing custom tags for the combination of title and abstract only
df['input_title_abstract'] = '<title>' + df['title'] + '</title>' + \
                             '<abstract>' + df['abstract'] + '</abstract>'

In [8]:
df.head(5)

,pmid,journal_name,title,abstract,accepted_label,input_journal_title_abstract,input_title_abstract
0,18379746,Der Radiologe,[Pediatric stroke].,Stroke in childhood has gained increasingly mo...,Non-systematic-review,<journal>Der Radiologe</journal><title>[Pediat...,<title>[Pediatric stroke].</title><abstract>St...
1,24660674,Journal of consulting and clinical psychology,Treatment engagement and response to CBT among...,"In the current study, we compared measures of ...",Human-RCT-non-drug-intervention,<journal>Journal of consulting and clinical ps...,<title>Treatment engagement and response to CB...
2,20159133,Archives of physical medicine and rehabilitation,Relationship between perceived exertion and ph...,To investigate the strength of the relationshi...,Remaining,<journal>Archives of physical medicine and reh...,<title>Relationship between perceived exertion...
3,11781147,Biochimica et biophysica acta,Characterization of a missense mutation at his...,Genetic defects in pyruvate dehydrogenase comp...,Remaining,<journal>Biochimica et biophysica acta</journa...,<title>Characterization of a missense mutation...
4,31706919,Epilepsy & behavior : E&B,The role of P-glycoprotein (P-gp) and inwardly...,Sudden unexpected death in epilepsy (SUDEP) is...,Non-systematic-review,<journal>Epilepsy & behavior : E&B</journal><t...,<title>The role of P-glycoprotein (P-gp) and i...


### Load key for the OpenAI API 

In [9]:
def load_pass(file_path, key_to_find):
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split("=")
            if len(parts) == 2 and parts[0] == key_to_find:
                found_password = parts[1]
                break
    if found_password:
        print("Found password.")
        return found_password
    else:
        print("Password not found for key:", key_to_find)

Note: You need to create a credentials.txt file with the following content:  
OPENAI=sk-77QXXXXXXXXXXXXXXXXXXXXXXXXXXX  
replace the value after the = sign with your API key.  
Make sure the credentials.txt is added to .gitignore, you don't want to put your password on Git!

In [10]:
openai.api_key = load_pass("./credentials.txt", "OPENAI")

Found password.


In [11]:
client = OpenAI(api_key=openai.api_key)

### Query GPT

To change the task the model is solving, you need to change the text of the prompt and the content text of the system role.  
To change the GPT model used, you need to change the text of the model name when initiating the openai API.  
The function gets as input the input_raw_text, that will be text for information extraction or classification.

In [12]:
import time

DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-3.5-turbo"

def query_gpt(input_raw_text, prompt_text, gpt_model="gpt-3.5-turbo", temperature=0, max_retries=5, retry_delay=3):
    # CHANGE gpt_model to the desired model name, see https://platform.openai.com/docs/models (gpt-3.5-turbo and gpt-4-turbo-preview)
    
    # Add a delay at the beginning of the function to avoid overloading the API if there are multiple calls
    # time.sleep(10)  

    system_msg = f"""
    You are an expert assistant specialized in text classification of PubMed abstracts. """

    retries = 0
    while retries < max_retries:
        print("Trying to call OpenAI API...")
        try:
            completion = client.chat.completions.create(
                model=gpt_model,  
                response_format={"type": "json_object"},
                temperature=temperature,
                #max_tokens=2000,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": prompt_text + input_raw_text}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            # Handle API error, e.g., retry or log
            print(f"OpenAI API returned an error: {e}")
            time.sleep(retry_delay)  # Wait before retrying
            retries += 1

    raise RuntimeError("Max retries reached. Unable to complete the API call.")


In [13]:
# Define a function to apply GPT queries with a progress bar
def apply_gpt_with_progress(data_series, prompt_text, model="gpt-3.5-turbo"):
    results = []
    total_items = len(data_series)
    # Create a tqdm progress bar
    with tqdm(total=total_items, desc=f"Processing dataset") as pbar:
        for text in data_series:
            result = query_gpt(text, prompt_text, model)
            results.append(result)
            pbar.update(1)  # Update the progress bar

    return results

# Read prompts from file and query GPT

In [14]:
json_file_path = "./prompt_strategies_shirin.json"
# Load the JSON file
with open(json_file_path, 'r') as file:
    prompts_data = json.load(file)

In [15]:
# Take the first half of the dataframe
#sampled_df = df.head(len(df) // 2)
#sampled_df.head()  # Show first few lines of sampled_df
#print(sampled_df)  # Shows whole sampled_df

In [16]:
# Take the second half of the dataframe
sampled_df = df.tail(len(df) // 2)
sampled_df.head()  # Show first few lines of remaining_df
#print(sampled_df)  # Shows whole remaining_df

,pmid,journal_name,title,abstract,accepted_label,input_journal_title_abstract,input_title_abstract
998,19176415,"Psychiatric services (Washington, D.C.)",Treatment patterns for schizoaffective disorde...,This study compared background characteristics...,Remaining,"<journal>Psychiatric services (Washington, D.C...",<title>Treatment patterns for schizoaffective ...
999,31747776,Autism : the international journal of research...,A multiple-method review of accommodations to ...,The purpose of this study is to understand the...,Non-systematic-review,<journal>Autism : the international journal of...,<title>A multiple-method review of accommodati...
1000,23665943,The Journal of toxicological sciences,Biochemical evaluation of the neurotoxicity of...,One of the toxicities caused by 1-Methyl-4-phe...,Animal-other,<journal>The Journal of toxicological sciences...,<title>Biochemical evaluation of the neurotoxi...
1001,8773796,The British journal of clinical psychology,Does community residence mean more community c...,The focus of the debate over hospital versus c...,Remaining,<journal>The British journal of clinical psych...,<title>Does community residence mean more comm...
1002,22638859,Journal of molecular neuroscience : MN,Detection of autoantibodies and increased conc...,The aim of this study was to assess levels of ...,Remaining,<journal>Journal of molecular neuroscience : M...,<title>Detection of autoantibodies and increas...


In [17]:
# Sample 50 elements randomly in a reproducible way
#sampled_df = df.sample(n=50, random_state=1)
#sampled_df.head() #show first few lines of sampled_df
#print(sampled_df) #shows whole sampled_df

## Run different prompts over the data

In [18]:
# Add the IDs of Prompts that you want to test
prompt_ids_to_test = ["P6"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        sampled_df[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(sampled_df['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        sampled_df[f'gpt_predictions_{prompt_id}'] = sampled_df[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        sampled_df.to_csv(f"predictions/{model}_test50_outputs_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")
        

Skipping prompt P1
Skipping prompt P2
Skipping prompt P2_1
Skipping prompt P3
Skipping prompt P3_1
Skipping prompt P3_2
Skipping prompt P3_3
Skipping prompt P3_4
Skipping prompt P4
Skipping prompt P4_1
Skipping prompt P4_2
Skipping prompt P5


Processing dataset:   0%|                      | 0/998 [00:00<?, ?it/s]

Trying to call OpenAI API...


Processing dataset:   0%|              | 1/998 [00:00<12:41,  1.31it/s]

Trying to call OpenAI API...


...

Processing dataset: 100%|███████████▉| 997/998 [13:02<00:00,  1.55it/s]

Trying to call OpenAI API...


Processing dataset: 100%|████████████| 998/998 [13:02<00:00,  1.27it/s]

Skipping prompt P7
Skipping prompt P8
Skipping prompt P9
Skipping prompt P9_1
Skipping prompt P10
Skipping prompt P11
Skipping prompt P11_1
Skipping prompt P11_2
Skipping prompt P11_3
Skipping prompt P11_4
Skipping prompt P11_5
Skipping prompt P12



/var/folders/9j/9lnwvx7s27531pb469pbr08w0000gq/T/ipykernel_99098/1123602534.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(sampled_df['input_journal_title_abstract'], prompt_text, model)
/var/folders/9j/9lnwvx7s27531pb469pbr08w0000gq/T/ipykernel_99098/1123602534.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[f'gpt_predictions_{prompt_id}'] = sampled_df[f'gpt_predictions_{prompt_id}_raw'].apply(


In [19]:
#prompts_data #shows contents of prompt file it reads in

In [20]:
sampled_df.head()

,pmid,journal_name,title,abstract,accepted_label,input_journal_title_abstract,input_title_abstract,gpt_predictions_P6_raw,gpt_predictions_P6
998,19176415,"Psychiatric services (Washington, D.C.)",Treatment patterns for schizoaffective disorde...,This study compared background characteristics...,Remaining,"<journal>Psychiatric services (Washington, D.C...",<title>Treatment patterns for schizoaffective ...,"{\n ""gpt_label"": ""Non-systematic-review""\n}",Non-systematic-review
999,31747776,Autism : the international journal of research...,A multiple-method review of accommodations to ...,The purpose of this study is to understand the...,Non-systematic-review,<journal>Autism : the international journal of...,<title>A multiple-method review of accommodati...,"{\n ""gpt_label"": ""Non-systematic-review""\n}",Non-systematic-review
1000,23665943,The Journal of toxicological sciences,Biochemical evaluation of the neurotoxicity of...,One of the toxicities caused by 1-Methyl-4-phe...,Animal-other,<journal>The Journal of toxicological sciences...,<title>Biochemical evaluation of the neurotoxi...,"{\n ""gpt_label"": ""Animal-drug-intervention""\n}",Animal-drug-intervention
1001,8773796,The British journal of clinical psychology,Does community residence mean more community c...,The focus of the debate over hospital versus c...,Remaining,<journal>The British journal of clinical psych...,<title>Does community residence mean more comm...,"{\n ""gpt_label"": ""Non-systematic-review""\n}",Non-systematic-review
1002,22638859,Journal of molecular neuroscience : MN,Detection of autoantibodies and increased conc...,The aim of this study was to assess levels of ...,Remaining,<journal>Journal of molecular neuroscience : M...,<title>Detection of autoantibodies and increas...,"{\n ""gpt_label"": ""Remaining""\n}",Remaining


## Evaluate each prompt

In [21]:
labels = ["Human-systematic-review", "Human-RCT-drug-intervention", "Human-RCT-non-drug-intervention", "Human-RCT-non-intervention", "Human-case-report", "Human-non-RCT-drug-intervention", "Human-non-RCT-non-drug-intervention", "Animal-systematic-review", "Animal-drug-intervention", "Animal-non-drug-intervention", "Animal-other", "Non-systematic-review", "In-vitro-study", "Clinical-study-protocol", "Remaining"]

label_to_numerical = {label: i for i, label in enumerate(labels)}
label_to_numerical["label missing"] = -1

In [22]:
def map_label_to_numerical(label):
    # Check if label is a dictionary
    if isinstance(label, dict):
        # Extract the label with the highest score/probability
        highest_label = max(label, key=label.get)
        return label_to_numerical.get(highest_label, -1)
    else:
        # Directly map string labels to numerical IDs
        return label_to_numerical.get(label, -1)
        
# Convert accepted labels to numerical
sampled_df['accepted_label_numerical'] = sampled_df['accepted_label'].apply(lambda x: label_to_numerical.get(x, -1))

# Initialize a list to hold DataFrame for each report and summary statistics
report_dfs = []
summary_stats = []

# Iterate over each GPT prediction column
for prompt_id in prompt_ids_to_test:
    print("Evaluating ", prompt_id)
    prediction_col = f'gpt_predictions_{prompt_id}'
    
    # Map GPT predictions to numerical values
    sampled_df[f'{prediction_col}_numerical'] = sampled_df[prediction_col].apply(map_label_to_numerical)
    
    # Extract arrays for evaluation
    y_true = sampled_df['accepted_label_numerical'].values
    y_pred = sampled_df[f'{prediction_col}_numerical'].values
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    accuracy_balanced = balanced_accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0, labels=range(len(labels)), target_names=labels)
    
    # Create DataFrame from report
    report_df = pd.DataFrame(report).transpose()
    report_df['Prompt ID'] = prompt_id  # Add column to indicate the prompt ID
    report_dfs.append(report_df)
    
    # Extract summary statistics (average precision, recall, F1)
    summary = report_df.loc['weighted avg', ['precision', 'recall', 'f1-score']].to_dict()
    summary['Prompt ID'] = prompt_id
    summary_stats.append(summary)

# Combine all report DataFrames
all_reports_df = pd.concat(report_dfs)

# Create a summary table for average precision, recall, and F1-score
summary_df = pd.DataFrame(summary_stats)

/var/folders/9j/9lnwvx7s27531pb469pbr08w0000gq/T/ipykernel_99098/290829237.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df['accepted_label_numerical'] = sampled_df['accepted_label'].apply(lambda x: label_to_numerical.get(x, -1))
/var/folders/9j/9lnwvx7s27531pb469pbr08w0000gq/T/ipykernel_99098/290829237.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df[f'{prediction_col}_numerical'] = sampled_df[prediction_col].apply(map_label_to_numerical)
/Library/Frameworks/Python.framework/V

Evaluating  P6


In [23]:
summary_df

,precision,recall,f1-score,Prompt ID
0,0.603545,0.50652,0.511981,P6


In [24]:
all_reports_df.to_csv(f"evaluations/{model}_test50_per_class_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
summary_df.to_csv(f"evaluations/{model}_test50_summary_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails

In [25]:
all_reports_df

,precision,recall,f1-score,support,Prompt ID
Human-systematic-review,0.478261,0.785714,0.594595,28.0,P6
Human-RCT-drug-intervention,0.578947,0.846154,0.687500,13.0,P6
Human-RCT-non-drug-intervention,0.647059,0.647059,0.647059,17.0,P6
Human-RCT-non-intervention,0.000000,0.000000,0.000000,1.0,P6
Human-case-report,0.343949,0.870968,0.493151,62.0,P6
Human-non-RCT-drug-intervention,0.636364,0.474576,0.543689,59.0,P6
Human-non-RCT-non-drug-intervention,0.202312,0.500000,0.288066,70.0,P6
Animal-systematic-review,0.000000,0.000000,0.000000,0.0,P6
Animal-drug-intervention,0.583333,0.840000,0.688525,50.0,P6
Animal-non-drug-intervention,0.384615,0.454545,0.416667,11.0,P6
